# Introduction and Overview
For assignment 5, I am using the global baseline recommender system from Project 2 to practice using PySpark.  Apache Spark allows parallelization of jobs, allowing us to build scalable recommender systems when the datasets are too large to transform efficiently on a single machine.  It utilizes lazy evaluation, where it creates a streamlined plan before acting on transofmrrations so that only the necessary steps are performed for the desired outcome.  Transformations are executed when view and collect actions are triggered.

To show the "equivalent" functions in PySpark, I left much of the Pandas code as comments so that parallels can be drawn. 

The main steps of this assignment will be the following:

1.  Spark Set-Up in Jupyter Notebook
2.  Data Import and Pre-processing with Spark
3.  Implement Spark with the global baseline recommender from Project 2
4.  Conclusions/Further exploration

# Spark Set-Up
To use PySpark in Jupyter Notebook, the correct Java version must be installed.  With the current version of Spark (4.0.0), Java 17 and 21 are required.  I downloaded Java 17 here: https://adoptium.net/temurin/releases?version=17&os=windows&arch=any.  I created a new System variable in System Properties > Environment Variables, for JAVA_HOME which pointed to the folder path of the Java download.  I added a new path, pointing to %JAVA_HOME%\bin.  I checked to ensure the correct version is being used by running Java -version in CMD.  When this showed the newer version (Java 23), I used "where Java" to show all locations pointing to Java installs.  The first result was a path pointing to Java 23, so I uninstalled it and rebooted.  This installed the correct version.  

For Spark, I installed Spark 4.0.0 here: https://spark.apache.org/downloads.html and then extracted the .tgz file using a windows shellscript command (tar -xzf spark-4.0.0-bin-hadoop3.tgz).  I created a new System variable (SPARK_HOME) pointing to the location of this extracted file, and a new path (%SPARK_HOME%\bin).  After installing the python packages required (findspark, pyspark), I can launch Jupyter Notebook in admin mode from command prompt and I am able to use PySpark.

In [1]:
import findspark
from pyspark import SparkContext
import pyspark 
from pyspark.sql import SparkSession
import os

In [2]:
findspark.init()
sc = SparkContext.getOrCreate()

os.environ["SPARK_LOCAL_DIRS"] = "C:\\Temp\\spark-temp"

spark = SparkSession.builder.config("spark.driver.memory", "6g").config("spark.executor.memory", "6g").getOrCreate()

spark

# Data Import and Pre-Processing with Spark
First, I import relevant packages and the joke ratings dataset (final.csv).  In this case, final.csv must be stored in a local path for import with PySpark.  I create the data schema, which is contains all the Jokes as columns, with DoubleType() for the ratings.  Creating the schema allows me to name the columns appropriately, and specify the datatype of the rating.  If the schema was not defined, the data import would have picked StringType() for the ratings.

In [3]:
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.functions import monotonically_increasing_id, concat_ws, col, lit, mean, stddev, stddev_samp, count_distinct, when, split
from pyspark.sql.types import IntegerType,StringType,StructField,StructType, BooleanType, DoubleType, FloatType

In [4]:
# import the user ratings
#user_ratings = pd.read_csv("https://raw.githubusercontent.com/koonkimb/Data612/refs/heads/main/Project_2/data/final.csv", header = None) 
schema = StructType([
    StructField("joke_000", DoubleType()), 
	StructField("joke_001", DoubleType()), 
	StructField("joke_002", DoubleType()), 
	StructField("joke_003", DoubleType()), 
	StructField("joke_004", DoubleType()), 
	StructField("joke_005", DoubleType()), 
	StructField("joke_006", DoubleType()), 
	StructField("joke_007", DoubleType()), 
	StructField("joke_008", DoubleType()), 
	StructField("joke_009", DoubleType()), 
	StructField("joke_010", DoubleType()), 
	StructField("joke_011", DoubleType()), 
	StructField("joke_012", DoubleType()), 
	StructField("joke_013", DoubleType()), 
	StructField("joke_014", DoubleType()), 
	StructField("joke_015", DoubleType()), 
	StructField("joke_016", DoubleType()), 
	StructField("joke_017", DoubleType()), 
	StructField("joke_018", DoubleType()), 
	StructField("joke_019", DoubleType()), 
	StructField("joke_020", DoubleType()), 
	StructField("joke_021", DoubleType()), 
	StructField("joke_022", DoubleType()), 
	StructField("joke_023", DoubleType()), 
	StructField("joke_024", DoubleType()), 
	StructField("joke_025", DoubleType()), 
	StructField("joke_026", DoubleType()), 
	StructField("joke_027", DoubleType()), 
	StructField("joke_028", DoubleType()), 
	StructField("joke_029", DoubleType()), 
	StructField("joke_030", DoubleType()), 
	StructField("joke_031", DoubleType()), 
	StructField("joke_032", DoubleType()), 
	StructField("joke_033", DoubleType()), 
	StructField("joke_034", DoubleType()), 
	StructField("joke_035", DoubleType()), 
	StructField("joke_036", DoubleType()), 
	StructField("joke_037", DoubleType()), 
	StructField("joke_038", DoubleType()), 
	StructField("joke_039", DoubleType()), 
	StructField("joke_040", DoubleType()), 
	StructField("joke_041", DoubleType()), 
	StructField("joke_042", DoubleType()), 
	StructField("joke_043", DoubleType()), 
	StructField("joke_044", DoubleType()), 
	StructField("joke_045", DoubleType()), 
	StructField("joke_046", DoubleType()), 
	StructField("joke_047", DoubleType()), 
	StructField("joke_048", DoubleType()), 
	StructField("joke_049", DoubleType()), 
	StructField("joke_050", DoubleType()), 
	StructField("joke_051", DoubleType()), 
	StructField("joke_052", DoubleType()), 
	StructField("joke_053", DoubleType()), 
	StructField("joke_054", DoubleType()), 
	StructField("joke_055", DoubleType()), 
	StructField("joke_056", DoubleType()), 
	StructField("joke_057", DoubleType()), 
	StructField("joke_058", DoubleType()), 
	StructField("joke_059", DoubleType()), 
	StructField("joke_060", DoubleType()), 
	StructField("joke_061", DoubleType()), 
	StructField("joke_062", DoubleType()), 
	StructField("joke_063", DoubleType()), 
	StructField("joke_064", DoubleType()), 
	StructField("joke_065", DoubleType()), 
	StructField("joke_066", DoubleType()), 
	StructField("joke_067", DoubleType()), 
	StructField("joke_068", DoubleType()), 
	StructField("joke_069", DoubleType()), 
	StructField("joke_070", DoubleType()), 
	StructField("joke_071", DoubleType()), 
	StructField("joke_072", DoubleType()), 
	StructField("joke_073", DoubleType()), 
	StructField("joke_074", DoubleType()), 
	StructField("joke_075", DoubleType()), 
	StructField("joke_076", DoubleType()), 
	StructField("joke_077", DoubleType()), 
	StructField("joke_078", DoubleType()), 
	StructField("joke_079", DoubleType()), 
	StructField("joke_080", DoubleType()), 
	StructField("joke_081", DoubleType()), 
	StructField("joke_082", DoubleType()), 
	StructField("joke_083", DoubleType()), 
	StructField("joke_084", DoubleType()), 
	StructField("joke_085", DoubleType()), 
	StructField("joke_086", DoubleType()), 
	StructField("joke_087", DoubleType()), 
	StructField("joke_088", DoubleType()), 
	StructField("joke_089", DoubleType()), 
	StructField("joke_090", DoubleType()), 
	StructField("joke_091", DoubleType()), 
	StructField("joke_092", DoubleType()), 
	StructField("joke_093", DoubleType()), 
	StructField("joke_094", DoubleType()), 
	StructField("joke_095", DoubleType()), 
	StructField("joke_096", DoubleType()), 
	StructField("joke_097", DoubleType()), 
	StructField("joke_098", DoubleType()), 
	StructField("joke_099", DoubleType()), 
	StructField("joke_100", DoubleType()), 
	StructField("joke_101", DoubleType()), 
	StructField("joke_102", DoubleType()), 
	StructField("joke_103", DoubleType()), 
	StructField("joke_104", DoubleType()), 
	StructField("joke_105", DoubleType()), 
	StructField("joke_106", DoubleType()), 
	StructField("joke_107", DoubleType()), 
	StructField("joke_108", DoubleType()), 
	StructField("joke_109", DoubleType()), 
	StructField("joke_110", DoubleType()), 
	StructField("joke_111", DoubleType()), 
	StructField("joke_112", DoubleType()), 
	StructField("joke_113", DoubleType()), 
	StructField("joke_114", DoubleType()), 
	StructField("joke_115", DoubleType()), 
	StructField("joke_116", DoubleType()), 
	StructField("joke_117", DoubleType()), 
	StructField("joke_118", DoubleType()), 
	StructField("joke_119", DoubleType()), 
	StructField("joke_120", DoubleType()), 
	StructField("joke_121", DoubleType()), 
	StructField("joke_122", DoubleType()), 
	StructField("joke_123", DoubleType()), 
	StructField("joke_124", DoubleType()), 
	StructField("joke_125", DoubleType()), 
	StructField("joke_126", DoubleType()), 
	StructField("joke_127", DoubleType()), 
	StructField("joke_128", DoubleType()), 
	StructField("joke_129", DoubleType()), 
	StructField("joke_130", DoubleType()), 
	StructField("joke_131", DoubleType()), 
	StructField("joke_132", DoubleType()), 
	StructField("joke_133", DoubleType()), 
	StructField("joke_134", DoubleType()), 
	StructField("joke_135", DoubleType()), 
	StructField("joke_136", DoubleType()), 
	StructField("joke_137", DoubleType()), 
	StructField("joke_138", DoubleType()), 
	StructField("joke_139", DoubleType()), 
	StructField("joke_140", DoubleType()), 
	StructField("joke_141", DoubleType()), 
	StructField("joke_142", DoubleType()), 
	StructField("joke_143", DoubleType()), 
	StructField("joke_144", DoubleType()), 
	StructField("joke_145", DoubleType()), 
	StructField("joke_146", DoubleType()), 
	StructField("joke_147", DoubleType()), 
	StructField("joke_148", DoubleType()), 
	StructField("joke_149", DoubleType()), 
	StructField("joke_150", DoubleType()), 
	StructField("joke_151", DoubleType()), 
	StructField("joke_152", DoubleType()), 
	StructField("joke_153", DoubleType()), 
	StructField("joke_154", DoubleType()), 
	StructField("joke_155", DoubleType()), 
	StructField("joke_156", DoubleType()), 
	StructField("joke_157", DoubleType()), 
	StructField("joke_158", DoubleType()) 
	
])

user_ratings = spark.read.csv("final.csv", schema = schema, header = False)

Like in Project 2, I drop the first column which is the number of jokes rated per user.  Since PySpark dataframes do not have indices, I create a user_index column using monotonically_increasing_id() from pyspark.sql.functions.  I use unpivot to "melt" the dataframe from wide to long format.

In [5]:
#user_ratings_df = user_ratings.drop(user_ratings.columns[0],axis = 1) # drop the first column which is number of jokes rated
#user_ratings_df = user_ratings_df.transpose() # transposing the matrix switches columns and rows
#user_ratings_df = user_ratings_df.reset_index() # reset index so that columns start at 0 again
#user_ratings_df = user_ratings_df.drop('index',axis = 1) # drop the index column
#user_ratings_df = user_ratings_df.transpose() # revert back to original structure
#user_ratings_df = user_ratings_df.reset_index() # reset index so that we can melt df with index as the id_vars
#print(user_ratings_df)

user_ratings = user_ratings.drop('joke_000')
user_columns = user_ratings.columns
user_ratings = user_ratings.withColumn('user_index', monotonically_increasing_id()) # insert user ids as spark df do not have indices
#user_ratings_long = user_ratings_df.melt(id_vars='index', var_name='joke', value_name='rating') # change df structure from wide to long
user_ratings = user_ratings.unpivot(ids=["user_index"], values=user_columns, variableColumnName="Joke", valueColumnName="Rating")
user_ratings.show(n=5)

+----------+--------+------+
|user_index|    Joke|Rating|
+----------+--------+------+
|         0|joke_001|  99.0|
|         0|joke_002|  99.0|
|         0|joke_003|  99.0|
|         0|joke_004|  99.0|
|         0|joke_005|  99.0|
+----------+--------+------+
only showing top 5 rows


I dropped the user - joke ratings that were designated as "99" as "99" rating indicates unrated.  Then I split the dataset into a training and test set using randomSplit().  This process was easier in PySpark than in Pandas.

In [6]:
#user_ratings_dropped = user_ratings_long
#user_ratings_dropped['rating'] = user_ratings_dropped['rating'].replace(99,np.nan) # replace 99 with np.nan as entries with 99 rating were not rated
#user_ratings_dropped = user_ratings_dropped.dropna() # drop the NA records
user_ratings = user_ratings.filter(col('Rating') != 99)
#df_random = user_ratings_dropped.sample(frac = 1, random_state = 63) # randomize the records
#split_size = int(0.8*len(df_random)) # designate split size (80%)
#train = df_random[:split_size] # split dataset into 80% train and 20% test
#test = df_random[split_size:]
train_df, test_df = user_ratings.randomSplit([0.8,0.2], seed = 63)

I pivot back the database from long to wide format.  I do this by grouping the Joke Ratings by user_index and then pivoting the dataframe.

In [7]:
#pivot_train = train.pivot(index='index', columns='joke', values='rating') # pivot the training data back into original format with jokes as columns
pivot_train = train_df.groupBy('user_index').pivot('Joke').agg(F.mean('Rating'))
#pivot_test = test.pivot(index='index', columns='joke', values='rating') # pivot the testing data back into original format with jokes as columns
pivot_test = test_df.groupBy('user_index').pivot('Joke').agg(F.mean('Rating'))

I check to see if there are missing users.  This ended up being unnecessary as RMSE was later calculated using a join, which would have eliminated any records that do not "match" (i.e. do not have both a prediction and an actual value).

In [8]:
pivot_train.select(F.count_distinct("user_index")).show() # see if there are missing users... there are
pivot_test.select(F.count_distinct("user_index")).show()

+--------------------------+
|count(DISTINCT user_index)|
+--------------------------+
|                      7398|
+--------------------------+

+--------------------------+
|count(DISTINCT user_index)|
+--------------------------+
|                      5021|
+--------------------------+



I found the missing rows and inserted nulls in these rows for each column.  I then unioned this data with the training and test data, so that the training and test data contained all of the users.  Again, this was unnecessary due to the inner join later used to calculate RMSE.  

In [9]:
df_missing_rows = user_ratings.select('user_index').distinct().orderBy(col('user_index'))
for i in range(158):
    if i < 9:
        column_name = f"joke_00{i+1}"
    elif i >= 9 and i < 99:
        column_name = f"joke_0{i+1}"
    else:
        column_name = f"joke_{i+1}"
        # print(column_name)# Generate unique column names
    if column_name in pivot_train.columns:
        pass
    else:
        pivot_train = pivot_train.withColumn(column_name, lit(None).cast(DoubleType()))
    if column_name in pivot_test.columns:
        pass
    else:
        pivot_test = pivot_test.withColumn(column_name, lit(None).cast(DoubleType()))
    #print(column_name)
    df_missing_rows = df_missing_rows.withColumn(column_name, lit(None).cast(DoubleType()))
pivot_train = pivot_train.select(df_missing_rows.columns)
pivot_test = pivot_test.select(df_missing_rows.columns)
missing_rows_train = df_missing_rows.join(pivot_train, on=["user_index"], how="left_anti")
missing_rows_test = df_missing_rows.join(pivot_test, on=["user_index"], how="left_anti")
#print(len(pivot_train.columns))
#print(len(missing_rows_train.columns))
#print(pivot_train.columns)
#print(missing_rows_train.columns)
#print(pivot_test.columns)
pivot_train_union = pivot_train.union(missing_rows_train)
pivot_test_union = pivot_test.union(missing_rows_test)

In [10]:
pivot_train_union.select(F.count_distinct("user_index")).show() 
pivot_test_union.select(F.count_distinct("user_index")).show()

+--------------------------+
|count(DISTINCT user_index)|
+--------------------------+
|                      7699|
+--------------------------+

+--------------------------+
|count(DISTINCT user_index)|
+--------------------------+
|                      7699|
+--------------------------+



# Global Baseline using PySpark
I calculate the global baseline using the mean value of all known ratings in the training set.  I access the scalar value via collect()[0][0].

In [11]:
#train_means = np.nanmean(pivot_train_reindex.values) # get mean of all values in the df
train_means = train_df.select(mean('Rating')).collect()[0][0] #

A matrix of the above mean is created.  I can use the df_missing_rows dataframe to get the desired dataframe shape, and then fill all the NA values with the mean calculated above.

In [12]:
train_means_df = df_missing_rows
train_means_df = train_means_df.fillna(train_means)

User bias is calculated by taking the mean rating of each user and subtracting the average rating across all ratings.  I can do this by grouping by user_index and finding the mean rating.  

In [13]:
#user_bias = pivot_train_reindex.mean(axis=1) - train_means # find user bias by taking mean rating for each user
#print(user_bias)
user_bias = train_df.groupBy("user_index").mean("Rating").orderBy("user_index")
user_bias = user_bias.withColumnRenamed("avg(Rating)","user_bias")
user_bias.show()

+----------+-------------------+
|user_index|          user_bias|
+----------+-------------------+
|         0|                3.7|
|         2|-1.0574999999999999|
|         3| 1.2291666666666667|
|         4|0.36916666666666664|
|         5| 2.8948148148148154|
|         6|-0.9511494252873564|
|         7|  4.409999999999999|
|         8| 7.8549999999999995|
|         9|-0.5845454545454545|
|        10|-0.4254545454545456|
|        11|             0.7525|
|        12|             -0.732|
|        13|                4.0|
|        14|              -5.75|
|        15|  5.194444444444445|
|        16|              4.215|
|        17| 6.8100000000000005|
|        18|              3.814|
|        19|                0.0|
|        20|               5.46|
+----------+-------------------+
only showing top 20 rows


Joke bias is calculated by taking the mean rating of each joke and subtracting the average rating across all ratings. I do the same as above and group by Joke to get the mean Rating by Joke.  

In [14]:
#joke_bias = pivot_train_reindex.mean() - train_means # find joke bias by taking mean rating for each joke
#print(joke_bias)
joke_bias = train_df.groupBy("Joke").mean("Rating").orderBy("Joke")
joke_bias = joke_bias.withColumnRenamed("avg(Rating)","joke_bias")
joke_bias.show()

+--------+--------------------+
|    Joke|           joke_bias|
+--------+--------------------+
|joke_007|  -1.424780701754386|
|joke_008| -1.5945876288659788|
|joke_013|   -3.11254189944134|
|joke_015|   -3.18295019157088|
|joke_016| -2.1721457489878544|
|joke_017|0.004354838709677506|
|joke_018| -2.0270867768595036|
|joke_019|-0.43669007021063183|
|joke_021|    1.00725321888412|
|joke_022| -0.9524496644295297|
|joke_023|-0.10372623574144489|
|joke_024| -1.5156834532374104|
|joke_025|-0.15845890410958893|
|joke_026|-0.06906250000000005|
|joke_028|  0.6304893617021278|
|joke_029|  1.1942028985507251|
|joke_030|-0.10337962962962949|
|joke_032|  2.2082767191383583|
|joke_033| -2.0537058823529413|
|joke_034| -0.6903940886699503|
+--------+--------------------+
only showing top 20 rows


User and joke biases are taken into account by adding them to the matrix of average ratings. This is done for both the training and test set.  This was more difficult with PySpark than Pandas, as there was no function to add row-wise or column-wise.  Instead, I joined the user_bias dataframe to the train_means_df and then for each column, added the user_bias to the corresponding row.  For the Joke biases, I created a dictionary and added the specified column bias if the column existed in the dictionary.  Finally, I used "when" from pyspark.sql.functions (similar to case when... then... else... in SQL) to change the rating to 10 if greater than 10, or -10 if less than -10.  

In [15]:
#train_means_user_bias = train_means_df.add(user_bias, axis = 0)
#train_means_bias = train_means_df.add(joke_bias, axis = 1)
#train_means_bias[train_means_bias > 10] = 10 # cap maximum ratings at 10
#train_means_bias[train_means_bias < -10] = -10 # cap minimal ratings at -10
#print(train_means_bias)
train_means_bias = train_means_df.join(user_bias, on="user_index", how="inner")
joke_cols = [c for c in train_means_df.columns if c != "user_index"]
joke_bias_dict = {row["Joke"]: row["joke_bias"] for row in joke_bias.collect()}
#print(joke_bias_dict)
for c in joke_cols:
    train_means_bias = train_means_bias.withColumn(c, col(c) + col("user_bias"))

    if c in joke_bias_dict:
        train_means_bias = train_means_bias.withColumn(c, col(c) + lit(joke_bias_dict[c]))
    train_means_bias = train_means_bias.withColumn(c, when(col(c) > 10, 10).otherwise(col(c)))
    train_means_bias = train_means_bias.withColumn(c, when(col(c) < -10, -10).otherwise(col(c)))
    
train_means_bias = train_means_bias.drop("user_bias")
train_means_bias.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+----------------

In [16]:
test_means_df = train_means_df
test_means_bias = test_means_df.join(user_bias, on="user_index", how="inner")
joke_cols = [c for c in test_means_df.columns if c != "user_index"]
joke_bias_dict = {row["Joke"]: row["joke_bias"] for row in joke_bias.collect()}
#print(joke_bias_dict)
for c in joke_cols:
    test_means_bias = test_means_bias.withColumn(c, col(c) + col("user_bias"))

    if c in joke_bias_dict:
        test_means_bias = test_means_bias.withColumn(c, col(c) + lit(joke_bias_dict[c]))
    test_means_bias = test_means_bias.withColumn(c, when(col(c) > 10, 10).otherwise(col(c)))
    test_means_bias = test_means_bias.withColumn(c, when(col(c) < -10, -10).otherwise(col(c)))

test_means_bias = test_means_bias.drop("user_bias")
test_means_bias.show()

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+----------------

I can then compute the RMSE by "melting" (i.e. unpivoting) the training dataframe and dropping NAs.  I do the same for the global baseline prediction dataframe, and then join the two dataframes together based on user_index and joke.  This gives me a dataframe in long format containing unique user_index, joke combinations along with the actual ratings and predicted value based on the global baseline prediction.  With this format of PySpark dataframe, I can use RegressionEvaluator from pyspark.ml.evaluation and indicate that the prediction column is in the column "Prediction" while the actual value is in the column "Rating".

In [17]:
rmse_bias_train = pivot_train_union.unpivot(ids=["user_index"], values=user_columns, variableColumnName="Joke", valueColumnName="Rating")
rmse_bias_train = rmse_bias_train.dropna()
train_means_bias_pivot = train_means_bias.unpivot(ids=["user_index"], values=user_columns, variableColumnName="Joke", valueColumnName="Rating")
train_means_bias_pivot = train_means_bias_pivot.withColumnRenamed("Rating","Prediction")
rmse_bias_train = rmse_bias_train.join(train_means_bias_pivot, on = ["user_index","Joke"], how = "inner")
rmse_bias_train.show()

+----------+--------+------+--------------------+
|user_index|    Joke|Rating|          Prediction|
+----------+--------+------+--------------------+
|        29|joke_072| -2.97|0.007858951056398444|
|      2250|joke_110|   1.1|   3.711327734783351|
|      2529|joke_127|  -2.3|  1.6611028975163178|
|      2927|joke_098|  10.0|  2.5021575379977863|
|      2927|joke_150|   0.0|   4.857877955631197|
|      3764|joke_047|  2.72|   3.188337505272332|
|      6721|joke_056|   0.0|  0.5623124545098462|
|      1258|joke_071| -0.33|  -0.709487306028212|
|      1409|joke_055| -7.87| -0.9481808735906244|
|      1409|joke_132| -9.94|-0.13808812342044047|
|      3155|joke_018|  -3.3| -3.4578110570435348|
|      3155|joke_152| -10.0| -1.2541126296985943|
|      1690|joke_019| -1.12|  -4.067083915612054|
|      2944|joke_019| -3.56|   -0.70038101706133|
|      6508|joke_138|  5.12| 0.30366995756821646|
|      1077|joke_104|   0.0|  3.2511191717463244|
|      4117|joke_105|  2.07|  0.3494399770575314|


In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol="Prediction", labelCol="Rating", metricName="rmse")
rmse_gb_train = evaluator.evaluate(rmse_bias_train)
print(rmse_gb_train)

4.28245951030375


In [19]:
rmse_bias_test = pivot_test_union.unpivot(ids=["user_index"], values=user_columns, variableColumnName="Joke", valueColumnName="Rating")
rmse_bias_test = rmse_bias_test.dropna()
test_means_bias_pivot = test_means_bias.unpivot(ids=["user_index"], values=user_columns, variableColumnName="Joke", valueColumnName="Rating")
test_means_bias_pivot = test_means_bias_pivot.withColumnRenamed("Rating","Prediction")
rmse_bias_test = rmse_bias_test.join(test_means_bias_pivot, on = ["user_index","Joke"], how = "inner")
rmse_bias_test.show()

+----------+--------+------+-------------------+
|user_index|    Joke|Rating|         Prediction|
+----------+--------+------+-------------------+
|      5572|joke_019|   0.0|0.09378564960533681|
|      6262|joke_088|  0.22| 1.5222981864302647|
|      6479|joke_062|  4.72|  2.228610943371039|
|      4636|joke_072|   0.0| 0.9894065701040174|
|      2961|joke_019| -6.77|-0.3270654142244505|
|      5957|joke_105|  1.63| 2.9869399770575313|
|      3787|joke_053|   6.6|  5.709307200581844|
|      3689|joke_054|  1.54| 2.3721281065968745|
|      5453|joke_118|  10.0| 5.3025372772182475|
|      2308|joke_127|   0.0|   2.20642810076835|
|      2841|joke_019|  2.64| 0.4657856496053368|
|      1969|joke_123| -4.88| 2.5165926029328523|
|      2153|joke_019|  4.39|-0.5459113200916328|
|      1002|joke_129| -1.89| 1.5110865417425785|
|      3343|joke_072|  5.98|0.31035895105639844|
|      2404|joke_132| -4.94|-5.0008738377061555|
|      2546|joke_132|   0.0|  5.050376162293844|
|      2414|joke_114

In [20]:
rmse_gb_test = evaluator.evaluate(rmse_bias_test)
print(rmse_gb_test)

4.712638333379371


# Conclusion

This exercise was intended to provide an introduction to PySpark, although its use in this assignment was not intended to demonstrate its utility given the small dataset.  I had first attempted to perform this exercise on Assignment 4, given its much larger dataset.  However, because I am running PySpark on my local machine and not a machine cluster (given that databricks no longer allows us to spin up clusters for free), I ran into significant memory issues.  Even when using the same pared down dataset in Assignment 4 (I had used only 20% of the dataset), PySpark encountered memory issues in areas where Pandas was able to complete equivalent transformations.  Given the limitations of my local machine, I had to swap to adapting Assignment 2 to PySpark.  

I ran into similar problems when attempting the Content-Based filtering step of my Assignment 2 recommender system, as shown below.  While I was able to find the features of the jokes dataset with TF-IDF in PySpark, using HashingTF and IDF in pyspark.ml.feature, my Spark session crashed continuously when attempting to calculate cosine similarity through a user defined function.  

While this exercise was a good introduction for manipulating Spark dataframes and provided an overview of its use and set-up, ultimately for smaller datasets and local machines, Pandas is likely the better choice.  However, I still believe that my Assignment 4 recommender system demonstrates the necessity of moving to a distributed platform.  At its full size, I encountered memory issues that prohibited me from transforming the data.  With access to machine clusters, these operations could have been performed given its access to more memory and parallelized processing.  For this current assignment, based off of assignment 2, the dataset is much smaller and can be fully implemented using just Pandas.  Many operations - such as finding cosine similarity, or any other element-wise operations - were more difficult using PySpark and required iteration.  However, the utility of SQL-like functions did streamline other transformations, such as finding distinct_count.  With exposure to more PySpark packages, such as RegressionEvaluator, it is likely that I can find better ways to perform the cumbersome operations.  

# Appendix: TF-IDF with PySpark

In [21]:
# Import the data
#jokes_df = pd.read_csv("https://raw.githubusercontent.com/koonkimb/Data612/refs/heads/main/Project_2/data/Dataset4JokeSet.csv", header = None)
jokes_df = spark.read.csv("Dataset4JokeSet.csv", header = False)

jokes_df = jokes_df.withColumn("Joke_Content",  concat_ws(" ", col("_c0"), col("_c1")))
jokes_df = jokes_df.withColumn('_c0', monotonically_increasing_id() + 1)

joke_index = train_means_bias_pivot.select("Joke").distinct().orderBy("Joke")
joke_index = joke_index.withColumn('_c0', monotonically_increasing_id() + 1)
jokes_df = joke_index.join(jokes_df, on = '_c0', how = 'inner')
jokes_df = jokes_df.drop('_c0','_c1')
jokes_df.printSchema()
jokes_df.show()

root
 |-- Joke: string (nullable = false)
 |-- Joke_Content: string (nullable = false)

+--------+--------------------+
|    Joke|        Joke_Content|
+--------+--------------------+
|joke_001|"A man visits the...|
|joke_002|"This couple had ...|
|joke_003|Q. What's 200 fee...|
|joke_004|Q. What's the dif...|
|joke_005|Q.\tWhat's O. J. ...|
|joke_006|Bill & Hillary ar...|
|joke_007|How many feminist...|
|joke_008|Q. Did you hear a...|
|joke_009|"A country guy go...|
|joke_010|"Two cannibals ar...|
|joke_011|Q. What do a hurr...|
|joke_012|"A guy stood over...|
|joke_013|"They asked the J...|
|joke_014|"The father was v...|
|joke_015|Q:  What did the ...|
|joke_016|Q. What is orange...|
|joke_017|How many men does...|
|joke_018|"A dog walks into...|
|joke_019|"Q: If a person w...|
|joke_020|What's the differ...|
+--------+--------------------+
only showing top 20 rows


In [22]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.linalg import DenseVector, Vectors
from pyspark.sql.functions import udf, col

In [23]:
joke_df_processed = jokes_df.withColumn("Joke_Content", split(jokes_df["Joke_Content"], " "))
hashingTF = HashingTF(inputCol="Joke_Content", outputCol="features")
joke_features_df = hashingTF.transform(joke_df_processed)
joke_features_df.show()

+--------+--------------------+--------------------+
|    Joke|        Joke_Content|            features|
+--------+--------------------+--------------------+
|joke_001|["A, man, visits,...|(262144,[3856,722...|
|joke_002|["This, couple, h...|(262144,[17893,18...|
|joke_003|[Q., What's, 200,...|(262144,[32769,41...|
|joke_004|[Q., What's, the,...|(262144,[3834,942...|
|joke_005|[Q.\tWhat's, O., ...|(262144,[61321,87...|
|joke_006|[Bill, &, Hillary...|(262144,[2665,119...|
|joke_007|[How, many, femin...|(262144,[19736,27...|
|joke_008|[Q., Did, you, he...|(262144,[18700,27...|
|joke_009|["A, country, guy...|(262144,[921,1199...|
|joke_010|["Two, cannibals,...|(262144,[16108,21...|
|joke_011|[Q., What, do, a,...|(262144,[19546,27...|
|joke_012|["A, guy, stood, ...|(262144,[5680,275...|
|joke_013|["They, asked, th...|(262144,[9378,253...|
|joke_014|["The, father, wa...|(262144,[9056,216...|
|joke_015|[Q:, , What, did,...|(262144,[19208,24...|
|joke_016|[Q., What, is, or...|(262144,[49331,

In [24]:
idf = IDF(inputCol="features", outputCol="IDF_features")
idf_model = idf.fit(joke_features_df)
tfidf_df = idf_model.transform(joke_features_df)
tfidf_df = tfidf_df.select("Joke","IDF_features")
tfidf_df.show()

+--------+--------------------+
|    Joke|        IDF_features|
+--------+--------------------+
|joke_001|(262144,[3856,722...|
|joke_002|(262144,[17893,18...|
|joke_003|(262144,[32769,41...|
|joke_004|(262144,[3834,942...|
|joke_005|(262144,[61321,87...|
|joke_006|(262144,[2665,119...|
|joke_007|(262144,[19736,27...|
|joke_008|(262144,[18700,27...|
|joke_009|(262144,[921,1199...|
|joke_010|(262144,[16108,21...|
|joke_011|(262144,[19546,27...|
|joke_012|(262144,[5680,275...|
|joke_013|(262144,[9378,253...|
|joke_014|(262144,[9056,216...|
|joke_015|(262144,[19208,24...|
|joke_016|(262144,[49331,83...|
|joke_017|(262144,[14273,27...|
|joke_018|(262144,[4631,252...|
|joke_019|(262144,[70488,80...|
|joke_020|(262144,[3834,722...|
+--------+--------------------+
only showing top 20 rows


In [25]:
spark.stop()